In [ ]:
import numpy as np
import xarray as xr
import os,errno
import sys
import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt
import salem
import scipy.stats as st
import warnings
warnings.filterwarnings('ignore')

dir_data='/Users/ahenny/'
dir2='/Volumes/Extreme Pro/'
dir3='/Volumes/My Passport/'

In [ ]:
#to check results
import cartopy.crs as ccrs
ds=xr.open_dataset(dir2+'merra2.ar.labels.model.'+str(2023)+'.nc')
test=ds['ar_labeled'][180*2,:,:]

fig=plt.figure(figsize=(20,10))
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
ax.pcolormesh(test.lon,test.lat,test,vmin=0,vmax=test.max().values,cmap=plt.cm.seismic)
ax.coastlines(resolution='10m')
plt.show()

In [ ]:
import cartopy.crs as ccrs
ds=xr.open_dataset(dir2+'merra2.ar.labels.model.poleward.'+str(2023)+'.nc')
test=ds['ar_labeled'][180,:,:]

fig=plt.figure(figsize=(20,10))
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
ax.pcolormesh(test.lon,test.lat,test.where(test>0),vmin=0,vmax=test.max().values,cmap=plt.cm.seismic)
ax.coastlines(resolution='10m')
plt.show()

In [ ]:
yrs=np.arange(2023,2024,1)
for i in range(len(yrs)):
    year=yrs[i]
    print(year)
    elim_count=0
    ds=xr.open_dataset(dir2+'merra2.ar.labels.model.'+str(year)+'.nc')
    labeled=ds['ar_labeled'][0::2,:,:]
    weights=np.cos(np.deg2rad(labeled.lat))
    
    if i==0:
        lat_array=xr.zeros_like(labeled[0,:,:])
        lon_array=xr.zeros_like(labeled[0,:,:])
        for k in range(labeled.lon.size):
            lat_array[:,k]=labeled.lat.values
        for k in range(labeled.lat.size):
            lon_array[k,:]=labeled.lon.values
    
    #ds=xr.open_dataset(dir2+'era_5_ivtv_'+str(year)+'.nc')
    ds=xr.open_dataset(dir2+'merra2.ivt.3hr.model.2023.nc')
    ivtv=ds['ivtv'][0::2,:,:]

    for j in range(labeled.time.size):
        if j%100==0:
            print(j)
        labeled_sel=labeled[j,:,:]
        
        #ivtv_sel=ds['p72.162'][j,:,:]
        #lons_east=[x for x in ivtv_sel.longitude.values if x>=180]
        #lons_west=[x for x in ivtv_sel.longitude.values if x<180]
        #field_east=ivtv_sel.sel(longitude=lons_east)
        #field_west=ivtv_sel.sel(longitude=lons_west)
        #field_east['longitude']=[x-360. for x in lons_east]
        #ivtv_sel=xr.concat([field_east,field_west],dim='longitude')        
        
        #ivtv_sel=ds['ivt'][j,:,:]
        ivtv_sel=ivtv[j,:,:]
        
        #dk=xr.Dataset()
        #dk['ivtv']=(('lat','lon'),ivtv_sel.values)
        #dk.coords['lat']=ivtv_sel.latitude.values
        #dk.coords['lon']=ivtv_sel.longitude.values
        #ivtv_sel=dk['ivtv']
        
        ivtv_interp=ivtv_sel.interp_like(labeled_sel)
        #ivtv_interp=ds['ivtv'][j,:,:]#JRA-55 - float lat/lon issues
        ivtv_interp['lat']=labeled_sel.lat.values
        ivtv_interp['lon']=labeled_sel.lon.values
        
        unique=np.unique(labeled_sel.values)
        unique=[x for x in unique if x>0]
        labeled_final=xr.zeros_like(labeled_sel)
        for k in unique:
            feature_sel=labeled_sel.where(labeled_sel==k).fillna(0)
            
            lats_sel=lat_array.where(labeled_sel==k)
            lats_weighted=lats_sel.weighted(weights)
            lat_mean=lats_weighted.mean(skipna=True).values.tolist()
            
            ivtv_sel=ivtv_interp.where(labeled_sel==k)
            ivtv_weighted=ivtv_sel.weighted(weights)
            ivtv_mean=ivtv_weighted.mean(skipna=True).values.tolist()
            
            if lat_mean>0 and ivtv_mean>=50.:
                labeled_final=labeled_final+feature_sel
            elif lat_mean<0 and ivtv_mean<=-50:
                labeled_final=labeled_final+feature_sel
            else:
                elim_count=elim_count+1
                
        if j==0:
            labeled_concat=labeled_final
        else:
            labeled_concat=xr.concat([labeled_concat,labeled_final],dim='time')
            
    print(elim_count)
    dk=xr.Dataset()
    dk['ar_labeled']=(('time','lat','lon'),labeled_concat.values)
    dk.coords['time']=labeled_concat.time
    dk.coords['lat']=labeled_concat.lat
    dk.coords['lon']=labeled_concat.lon
    try:
        os.remove(dir2+'merra2.ar.labels.model.poleward.'+str(year)+'.nc')
    except OSError:
        pass
    dk.to_netcdf(dir2+'merra2.ar.labels.model.poleward.'+str(year)+'.nc',mode='w',format='NETCDF4')
    dk.close()

In [ ]:
yrs=np.arange(1980,2001,1)
for i in range(len(yrs)):
    year=yrs[i]
    print(year)
    elim_count=0
    ds=xr.open_dataset(dir2+'merra2.ar.labels.model.'+str(year)+'.nc')
    labeled=ds['ar_labeled'][0::2,:,:]
    weights=np.cos(np.deg2rad(labeled.lat))
    
    if i==0:
        lat_array=xr.zeros_like(labeled[0,:,:])
        lon_array=xr.zeros_like(labeled[0,:,:])
        for k in range(labeled.lon.size):
            lat_array[:,k]=labeled.lat.values
        for k in range(labeled.lat.size):
            lon_array[k,:]=labeled.lon.values
    
    ds=xr.open_dataset(dir2+'merra2.ivt.3hr.model.'+str(year)+'.nc')
    ivtv=ds['ivtv'][0::2,:,:]
    
    for j in range(labeled.time.size):
        if j%100==0:
            print(j)
        labeled_sel=labeled[j,:,:]
        ivtv_sel=ivtv[j,:,:]
        
        unique=np.unique(labeled_sel.values)
        unique=[x for x in unique if x>0]
        labeled_final=xr.zeros_like(labeled_sel)
        for k in unique:
            feature_sel=labeled_sel.where(labeled_sel==k).fillna(0)
            
            lats_sel=lat_array.where(labeled_sel==k)
            lats_weighted=lats_sel.weighted(weights)
            lat_mean=lats_weighted.mean(skipna=True).values.tolist()
            
            ivtv_sel1=ivtv_sel.where(labeled_sel==k)
            ivtv_weighted=ivtv_sel1.weighted(weights)
            ivtv_mean=ivtv_weighted.mean(skipna=True).values.tolist()
            
            if lat_mean>0 and ivtv_mean>=50.:
                labeled_final=labeled_final+feature_sel
            elif lat_mean<0 and ivtv_mean<=-50:
                labeled_final=labeled_final+feature_sel
            else:
                elim_count=elim_count+1
                
        if j==0:
            labeled_concat=labeled_final
        else:
            labeled_concat=xr.concat([labeled_concat,labeled_final],dim='time')
            
    dk=xr.Dataset()
    dk['ar_labeled']=(('time','lat','lon'),labeled_concat.values)
    dk.coords['time']=labeled_concat.time
    dk.coords['lat']=labeled_concat.lat
    dk.coords['lon']=labeled_concat.lon
    try:
        os.remove(dir2+'merra2.ar.labels.model.poleward.'+str(year)+'.nc')
    except OSError:
        pass
    dk.to_netcdf(dir2+'merra2.ar.labels.model.poleward.'+str(year)+'.nc',mode='w',format='NETCDF4')
    dk.close()